In [ ]:
-- Parse content from DOCX files
WITH files AS (
  SELECT 
    REPLACE(REGEXP_SUBSTR(file_url, '[^/]+$'), '%2e', '.') as filename
  FROM DIRECTORY('@avalanche_db.avalanche_schema.avalanche_stage')
  WHERE filename LIKE '%.docx'
)
SELECT 
  filename,
  SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
    @avalanche_db.avalanche_schema.avalanche_stage,
    filename,
    {'mode': 'layout'}
  ):content AS content
FROM files;

In [ ]:
-- Extract PRODUCT name, DATE, and CUSTOMER_REVIEW from the LAYOUT column
SELECT
    filename,
    REGEXP_SUBSTR(content, 'Product: (.*?) Date:', 1, 1, 'e') as product,
    REGEXP_SUBSTR(content, 'Date: (202[0-9]-[0-9]{2}-[0-9]{2})', 1, 1, 'e') as date,
    CASE 
        WHEN POSITION('Customer Review' IN content) > 0 THEN
            SUBSTRING(content, POSITION('Customer Review' IN content) + LENGTH('Customer Review'))
        ELSE NULL
    END as customer_review
FROM {{cell1}};

In [ ]:
-- Create a table to store the parsed and structured document content with order_id
CREATE OR REPLACE TABLE avalanche_db.avalanche_schema.parsed_reviews (
    filename VARCHAR,
    order_id INTEGER,
    product VARCHAR,
    review_date DATE,
    customer_review TEXT
);

-- Insert the parsed content, including order_id derived from filename
INSERT INTO avalanche_db.avalanche_schema.parsed_reviews (filename, order_id, product, review_date, customer_review)
SELECT
    filename,
    -- Extract numeric portion from filename, prefix it with '1', and convert it to integer
    CAST(CONCAT('1', REGEXP_SUBSTR(filename, '\\d+')) AS INTEGER) AS order_id,
    product,
    TO_DATE(date) as review_date,
    customer_review
FROM {{cell2}};

In [ ]:
SELECT * FROM AVALANCHE_DB.AVALANCHE_SCHEMA.PARSED_REVIEWS;